# Incoming particles

In this notebook we are going to quantify the pathways of incoming particles as a function of incoming height and target organism

First we define the simulations we are going to analyse by defining some important parameters. These must be compatible with the particle simulations on your machine.

In [1]:
foldernames = ['21objects','16objects']
flowset = ['parabolic','waveparabolic']
tstep = '001'
fb = 'forward'
d2cmax = 0.05
beaching_strategy = '2'
perarea = True

Next we are going to import the relevant modules

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
import cmocean
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.lines import Line2D

Given the parameters chosen above, the data of incoming particle simulations are opened including some ranges and grids to use in the figures later

In [3]:
filename = flowset[0]+'.nc'
flowdata = xr.open_dataset(foldernames[0]+'/'+filename)

dx = flowdata['X'][1]-flowdata['X'][0]
dy = dx

objects0 = np.load(foldernames[0]+'/preprocessed/'+'objects.npy')

objects1 = np.load(foldernames[1]+'/preprocessed/'+'objects.npy')
x,y = np.meshgrid(flowdata['X'],flowdata['Y'])
xmesh,ymesh = np.meshgrid(np.arange(flowdata['X'].values[0]-0.5*dx, flowdata['X'].values[-1]+0.5*dx, dx),
                np.arange(flowdata['Y'].values[0]+0.5*dy, flowdata['Y'].values[-1]-1.5*dy, -dy))
xob,yob = np.meshgrid(np.arange(-0.5, len(objects0)+0.5, 1),
                np.arange(-0.5, len(objects0)+0.5, 1))
xfs,yfs = np.meshgrid(np.arange(-0.5, len(flowset)+0.5, 1),
                np.arange(-0.5, len(flowset)+0.5, 1))
labellist = [0, 1, 3, 4, 6, 7, 9, 10, 11, 12, 13, 15, 16, 17, 19, 21]                            # The numbers of the objects that remain in the run with certain objects left out
my_cmap = ListedColormap(['dodgerblue','g','purple','darkorange','red','purple'])

The volume flux from the boundary towards each of the objects is calculated in postprocessing and saved in a pickle file. Here we open these files and save them in the variable `freshmatrix`

In [11]:
freshmatrix = np.zeros((len(flowset)*len(foldernames),len(objects0)))
for i,flow in enumerate(flowset):
    if perarea:
        freshmatrix[2*i] = np.load(foldernames[0] + '/postprocessed/freshmatrix-Repeat-Perarea-' + beaching_strategy + '-' + flow + '-' + tstep + '-' + str(d2cmax)+'.npy')
        freshmatrix[2*i+1,labellist] = np.load(foldernames[1] + '/postprocessed/freshmatrix-Repeat-Perarea-' + beaching_strategy + '-' + flow + '-' + tstep + '-' + str(d2cmax)+'.npy')
    else:
        freshmatrix[2*i] = np.load(foldernames[0] + '/postprocessed/freshmatrix-Repeat-' + beaching_strategy + '-' + flow + '-' + tstep + '-' + str(d2cmax)+'.npy')
        freshmatrix[2*i+1,labellist] = np.load(foldernames[1] + '/postprocessed/freshmatrix-Repeat-' + beaching_strategy + '-' + flow + '-' + tstep + '-' + str(d2cmax)+'.npy')


In this study we are interested in the effects of waves and the effects of spacing on our transport pathways. To show this we are going to plot volume flux to each object, comparing first the different flow regimes and next the different spacings

In [25]:
%matplotlib qt
fig = plt.figure(figsize=(18,7.5))

heights = [3, 1]
spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=heights, hspace=0.25)

ax0 = fig.add_subplot(spec[0])
ax0.grid()

xlocs = np.zeros(len(objects1))
for i in range(len(objects1)):
    xlocs[i] = np.mean(x[objects1[i]])
    
ax0.bar(xlocs,freshmatrix[1,labellist],width=0.15,label=flowset[0],color=my_cmap(2))
ax0.bar(xlocs+0.05,freshmatrix[3,labellist],width=0.15,label=flowset[1],color=my_cmap(3))
ax0.set_axisbelow(True)

ax0.set_xticks(xlocs)
ax0.set_xticklabels(labellist)
ax0.set_xlim(0.3,8.5)
ax0.set_xlabel("ending object",fontsize=18)
if perarea:
    ax0.set_ylabel("Volume flux [$m$ $s^{-1}$]",fontsize = 18)
else:
    ax0.set_ylabel("Volume flux $m^2$ $s^{-1}$",fontsize = 18)
ax0.legend(fontsize = 18)
ax0.tick_params(labelsize=16)

ax1 = fig.add_subplot(spec[1])
ax1.grid(axis='x')
ax1.set_xlim(0.3,8.5)


ax1.tick_params(      # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    top=False,         # ticks along the top edge are off
    left=False,      # ticks along the left edge are off
    labelleft=False) # labels along the left edge are off

ax1.set_xticks(xlocs)
ax1.set_xticklabels(labellist,fontsize=16)

for i in range(len(objects1)):
    ax1.scatter(x[objects1[i]],y[objects1[i]],s=10,c=['k'])

if perarea:
    plt.savefig('Figures/fresh-flowcomp-16-perarea')
else:
    plt.savefig('Figures/fresh-flowcomp-16')

In [26]:
%matplotlib qt
fig = plt.figure(figsize=(18,7.5))

heights = [3, 1]
spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=heights, hspace=0.25)

ax0 = fig.add_subplot(spec[0])
ax0.grid()

xlocs = np.zeros(len(objects0))
for i in range(len(objects0)):
    xlocs[i] = np.mean(x[objects0[i]])
    
ax0.bar(xlocs,freshmatrix[0],width=0.15,label=flowset[0],color=my_cmap(2))
ax0.bar(xlocs+0.05,freshmatrix[2],width=0.15,label=flowset[1],color=my_cmap(3))
ax0.set_axisbelow(True)

ax0.set_xticks(xlocs)
ax0.set_xticklabels(np.arange(0,len(objects0),1))
ax0.set_xlim(0.3,8.5)
ax0.set_xlabel("ending object",fontsize=18)
if perarea:
    ax0.set_ylabel("Volume flux [$m$ $s^{-1}$]",fontsize = 18)
else:
    ax0.set_ylabel("$m^2$ $s^{-1}$",fontsize = 18)
ax0.legend(fontsize = 18)
ax0.tick_params(labelsize=16)

ax1 = fig.add_subplot(spec[1])
ax1.grid(axis='x')
ax1.set_xlim(0.3,8.5)


ax1.tick_params(      # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    top=False,         # ticks along the top edge are off
    left=False,      # ticks along the left edge are off
    labelleft=False) # labels along the left edge are off

ax1.set_xticks(xlocs)
ax1.set_xticklabels(np.arange(0,len(objects0),1),fontsize=16)

for i in range(len(objects0)):
    ax1.scatter(x[objects0[i]],y[objects0[i]],s=10,c=['k'])

if perarea:
    plt.savefig('Figures/fresh-flowcomp-21-perarea')
else:
    plt.savefig('Figures/fresh-flowcomp-21')

And next the volume fluxes for the simulations with different object spacing.

In [27]:
fig = plt.figure(figsize=(18,7.5))

heights = [3, 1]
spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=heights, hspace=0.25)

ax0 = fig.add_subplot(spec[0])
ax0.grid()

xlocs0 = np.zeros(len(objects0))
for i in range(len(objects0)):
    xlocs0[i] = np.mean(x[objects0[i]])
    
xlocs1 = np.zeros(len(objects1))
for i in range(len(objects1)):
    xlocs1[i] = np.mean(x[objects1[i]])
    
ax0.bar(xlocs0,freshmatrix[2],width=0.15,label=foldernames[0],color=my_cmap(0))
ax0.bar(xlocs1+0.05,freshmatrix[3,labellist],width=0.15,label=foldernames[1],color=my_cmap(1))
ax0.set_axisbelow(True)

ax0.set_xticks(xlocs0)
ax0.set_xticklabels(np.arange(0,len(objects0),1))
ax0.set_xlim(0.3,8.5)
ax0.set_xlabel("ending object",fontsize=18)
if perarea:
    ax0.set_ylabel("Volume flux [$m$ $s^{-1}$]",fontsize = 18)
else:
    ax0.set_ylabel("$m^2$ $s^{-1}$",fontsize = 18)
ax0.legend(fontsize = 18)
ax0.tick_params(labelsize=16)

ax1 = fig.add_subplot(spec[1])
ax1.grid(axis='x')
ax1.set_xlim(0.3,8.5)

ax1.tick_params(      # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    top=False,         # ticks along the top edge are off
    left=False,      # ticks along the left edge are off
    labelleft=False) # labels along the left edge are off

ax1.set_xticks(xlocs0)
ax1.set_xticklabels(np.arange(0,len(objects0),1),fontsize=16)
for i in range(len(objects0)):
    ax1.scatter(x[objects0[i]],y[objects0[i]],s=10,c=['r'])
for i in range(len(objects1)):
    ax1.scatter(x[objects1[i]],y[objects1[i]],s=10,c=['k'])

custom_lines = [Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['Taken out'],loc='upper left',fontsize=14)

if perarea:
    plt.savefig('Figures/fresh-obscomp-wave-perarea')
else:
    plt.savefig('Figures/fresh-obscomp-wave')

In [28]:
fig = plt.figure(figsize=(18,7.5))

heights = [3, 1]
spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=heights, hspace=0.25)

ax0 = fig.add_subplot(spec[0])
ax0.grid()

xlocs0 = np.zeros(len(objects0))
for i in range(len(objects0)):
    xlocs0[i] = np.mean(x[objects0[i]])
    
xlocs1 = np.zeros(len(objects1))
for i in range(len(objects1)):
    xlocs1[i] = np.mean(x[objects1[i]])
    
ax0.bar(xlocs0,freshmatrix[0],width=0.15,label=foldernames[0],color=my_cmap(0))
ax0.bar(xlocs1+0.05,freshmatrix[1,labellist],width=0.15,label=foldernames[1],color=my_cmap(1))
ax0.set_axisbelow(True)

ax0.set_xticks(xlocs0)
ax0.set_xticklabels(np.arange(0,len(objects0),1))
ax0.set_xlim(0.3,8.5)
ax0.set_xlabel("ending object",fontsize=18)
if perarea:
    ax0.set_ylabel("Volume flux [$m$ $s^{-1}$]",fontsize = 18)
else:
    ax0.set_ylabel("$m^2$ $s^{-1}$",fontsize = 18)
ax0.legend(fontsize = 18)
ax0.tick_params(labelsize=16)

ax1 = fig.add_subplot(spec[1])
ax1.grid(axis='x')
ax1.set_xlim(0.3,8.5)

ax1.tick_params(      # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    top=False,         # ticks along the top edge are off
    left=False,      # ticks along the left edge are off
    labelleft=False) # labels along the left edge are off

ax1.set_xticks(xlocs0)
ax1.set_xticklabels(np.arange(0,len(objects0),1),fontsize=16)
for i in range(len(objects0)):
    ax1.scatter(x[objects0[i]],y[objects0[i]],s=10,c=['r'])
for i in range(len(objects1)):
    ax1.scatter(x[objects1[i]],y[objects1[i]],s=10,c=['k'])

custom_lines = [Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['Taken out'],loc='upper left',fontsize=14)

if perarea:
    plt.savefig('Figures/fresh-obscomp-perarea')
else:
    plt.savefig('Figures/fresh-obscomp')

From the animations of the incoming particles the height distributions were saved. These can be compared here

In [5]:
zinits = np.zeros((len(flowset)*len(foldernames),10))
d  = np.linspace(-0.5,0.5,11)          # Boundaries at which to draw the bars 

for i,flow in enumerate(flowset):
    zinits[2*i] = np.load(foldernames[0] + '/postprocessed/zinits' + beaching_strategy + '-' + flow + '-forward-60-' + str(d2cmax)[2:]+'.npy')
    zinits[2*i+1] = np.load(foldernames[1] + '/postprocessed/zinits' + beaching_strategy + '-' + flow + '-forward-60-' + str(d2cmax)[2:]+'.npy')

In [6]:
freshdata = data.where(data['d2c'] < d2cmax,drop=True)  # only particles that enter boundary layers
freshdata = freshdata.where(freshdata['closestobject']<len(objects),drop =True)
reached = []
for i in range(len(freshdata['traj'])):
    reached += [int(freshdata['closestobject'].isel(traj=i).dropna('obs')[0])]  # the first object that is reached
reached = np.array(reached)  # first object number for each trajectory
if len(objects) < 20:
    reached = np.array(labellist)[reached]

# `edata` is a subset of data with the entire trajectories of the particles that enter boundary layers.
# the amount of trajectories is the same as the length of `reached`
e = data.where(data['d2c'] < d2cmax, drop=True)
e = e.where(e['closestobject']<len(objects), drop=True)['trajectory']
e = np.nan_to_num(e, nan=-1)
e = np.unique(e)
edata = data.where(data['trajectory'].isin([e]), drop=True)

zm = edata['z'][edata['z'][:,-1].isnull()]
zz = np.zeros(len(zm))
for i in range(len(zz)):
    zx = zm[i][zm[i].notnull()]
    zz[i] = zx[-1]

zs,bineds = np.histogram(data['z'][:,0],np.linspace(np.min(-flowdata['Y'])-0.001,np.max(-flowdata['Y'])+0.001,nbins+1))
zzs=np.zeros((nbins,nbins))
for i in range(len(zzs)):
    mask = (bineds[i]<=zm[:,0]) & (zm[:,0]<=bineds[i+1])
    zzs[i,:],bineds = np.histogram(zz[mask],bineds)

NameError: name 'data' is not defined

In [10]:
%matplotlib qt
i = 0 #0 = parabolic, 1 = waveparabolic
fig = plt.figure(figsize = (8,8))
ax = plt.axes()
ax.barh(d[:-1],zinits[2*i+1],0.1,align='edge',color='g',label='16 objects',alpha=0.65)
ax.barh(d[:-1],zinits[2*i],0.1,align='edge',color='dodgerblue',label='21 objects',alpha=0.65)
ax.set_yticks(d)
ax.tick_params(labelsize=16)
ax.set_ylabel("Entering height (m)",fontsize=18)
ax.set_xlabel("Number of particles",fontsize=18)
ax.set_ylim(-0.5,0.5)
ax.set_xlim(0,3500)
# ax.set_title('Entering particles height distribution '+flowset[i],fontsize=16)
custom_lines = [Line2D([0], [0], color='dodgerblue', lw=4),
                Line2D([0], [0], color='g', lw=4)]
ax.legend(custom_lines, ['21 objects', '16 objects'],loc='upper left',fontsize=14)
plt.savefig('Figures/hdistcomp-'+ beaching_strategy + '-'+flowset[i]+ str(d2cmax)[2:])

In [16]:
i = 0 #0 = parabolic, 1 = waveparabolic
fig = plt.figure(figsize = (8,8))
ax = plt.axes()
ax.barh(d[:-1],zinits[2*i+1]/np.sum(zinits[2*i+1]),0.1,align='edge',color='g',label='16 objects',alpha=0.65)
ax.barh(d[:-1],zinits[2*i]/np.sum(zinits[2*i]),0.1,align='edge',color='dodgerblue',label='21 objects',alpha=0.65)
ax.set_yticks(d)
ax.tick_params(labelsize=12)
ax.set_ylabel("Entering height (m)",fontsize=15)
ax.set_xlabel("Fraction of particles released",fontsize=15)
ax.set_ylim(-0.5,0.5)
ax.set_title('Entering particles height distribution '+flowset[i],fontsize=16)
custom_lines = [Line2D([0], [0], color='dodgerblue', lw=4),
                Line2D([0], [0], color='g', lw=4)]
ax.legend(custom_lines, ['21 objects', '16 objects'],loc='upper left',fontsize=14)
plt.savefig('Figures/relhdistcomp-'+ beaching_strategy + '-'+flowset[i]+ str(d2cmax)[2:])

Now finally let's count all the incoming particles that reach each object to see how the total volume flux to each object changes

In [19]:
tvf = np.zeros((len(flowset)*len(foldernames),len(objects0)))
for i,flow in enumerate(flowset):
    if perarea:
        tvf[2*i] = np.load(foldernames[0] + '/postprocessed/totalvolumeflux-Repeat-Perarea-' + beaching_strategy + '-' + flow + '-' + tstep + '-' + str(d2cmax)+'.npy')
        tvf[2*i+1,labellist] = np.load(foldernames[1] + '/postprocessed/totalvolumeflux-Repeat-Perarea-' + beaching_strategy + '-' + flow + '-' + tstep + '-' + str(d2cmax)+'.npy')
    else:
        tvf[2*i] = np.load(foldernames[0] + '/postprocessed/totalvolumeflux-Repeat-' + beaching_strategy + '-' + flow + '-' + tstep + '-' + str(d2cmax)+'.npy')
        tvf[2*i+1,labellist] = np.load(foldernames[1] + '/postprocessed/totalvolumeflux-Repeat-' + beaching_strategy + '-' + flow + '-' + tstep + '-' + str(d2cmax)+'.npy')


In [29]:
%matplotlib qt
fig = plt.figure(figsize=(18,7.5))

heights = [3, 1]
spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=heights, hspace=0.25)

ax0 = fig.add_subplot(spec[0])
ax0.grid()

xlocs = np.zeros(len(objects1))
for i in range(len(objects1)):
    xlocs[i] = np.mean(x[objects1[i]])
    
ax0.bar(xlocs,tvf[1,labellist],width=0.15,label=flowset[0],color=my_cmap(2))
ax0.bar(xlocs+0.05,tvf[3,labellist],width=0.15,label=flowset[1],color=my_cmap(3))
ax0.set_axisbelow(True)

ax0.set_xticks(xlocs)
ax0.set_xticklabels(labellist)
ax0.set_xlim(0.3,8.5)
ax0.set_xlabel("ending object",fontsize=18)
if perarea:
    ax0.set_ylabel("Volume flux [$m$ $s^{-1}$]",fontsize = 18)
else:
    ax0.set_ylabel("$m^2$ $s^{-1}$",fontsize = 18)
ax0.legend(fontsize = 18)
ax0.tick_params(labelsize=16)

ax1 = fig.add_subplot(spec[1])
ax1.grid(axis='x')
ax1.set_xlim(0.3,8.5)


ax1.tick_params(      # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    top=False,         # ticks along the top edge are off
    left=False,      # ticks along the left edge are off
    labelleft=False) # labels along the left edge are off

ax1.set_xticks(xlocs)
ax1.set_xticklabels(labellist,fontsize=16)

for i in range(len(objects1)):
    ax1.scatter(x[objects1[i]],y[objects1[i]],s=10,c=['k'])

if perarea:
    plt.savefig('Figures/tvf-flowcomp-16-perarea')
else:
    plt.savefig('Figures/tvf-flowcomp-16')

In [30]:
%matplotlib qt
fig = plt.figure(figsize=(18,7.5))

heights = [3, 1]
spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=heights, hspace=0.25)

ax0 = fig.add_subplot(spec[0])
ax0.grid()

xlocs = np.zeros(len(objects0))
for i in range(len(objects0)):
    xlocs[i] = np.mean(x[objects0[i]])
    
ax0.bar(xlocs,tvf[0],width=0.15,label=flowset[0],color=my_cmap(2))
ax0.bar(xlocs+0.05,tvf[2],width=0.15,label=flowset[1],color=my_cmap(3))
ax0.set_axisbelow(True)

ax0.set_xticks(xlocs)
ax0.set_xticklabels(np.arange(0,len(objects0),1))
ax0.set_xlim(0.3,8.5)
ax0.set_xlabel("ending object",fontsize=18)
if perarea:
    ax0.set_ylabel("Volume flux [$m$ $s^{-1}$]",fontsize = 18)
else:
    ax0.set_ylabel("$m^2$ $s^{-1}$",fontsize = 18)
ax0.legend(fontsize = 18)
ax0.tick_params(labelsize=16)

ax1 = fig.add_subplot(spec[1])
ax1.grid(axis='x')
ax1.set_xlim(0.3,8.5)


ax1.tick_params(      # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    top=False,         # ticks along the top edge are off
    left=False,      # ticks along the left edge are off
    labelleft=False) # labels along the left edge are off

ax1.set_xticks(xlocs)
ax1.set_xticklabels(np.arange(0,len(objects0),1),fontsize=16)

for i in range(len(objects0)):
    ax1.scatter(x[objects0[i]],y[objects0[i]],s=10,c=['k'])

if perarea:
    plt.savefig('Figures/tvf-flowcomp-21-perarea')
else:
    plt.savefig('Figures/tvf-flowcomp-21')

In [31]:
fig = plt.figure(figsize=(18,7.5))

heights = [3, 1]
spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=heights, hspace=0.25)

ax0 = fig.add_subplot(spec[0])
ax0.grid()

xlocs0 = np.zeros(len(objects0))
for i in range(len(objects0)):
    xlocs0[i] = np.mean(x[objects0[i]])
    
xlocs1 = np.zeros(len(objects1))
for i in range(len(objects1)):
    xlocs1[i] = np.mean(x[objects1[i]])
    
ax0.bar(xlocs0,tvf[2],width=0.15,label=foldernames[0],color=my_cmap(0))
ax0.bar(xlocs1+0.05,tvf[3,labellist],width=0.15,label=foldernames[1],color=my_cmap(1))
ax0.set_axisbelow(True)

ax0.set_xticks(xlocs0)
ax0.set_xticklabels(np.arange(0,len(objects0),1))
ax0.set_xlim(0.3,8.5)
ax0.set_xlabel("ending object",fontsize=18)
if perarea:
    ax0.set_ylabel("Volume flux [$m$ $s^{-1}$]",fontsize = 18)
else:
    ax0.set_ylabel("$m^2$ $s^{-1}$",fontsize = 18)
ax0.legend(fontsize = 18)
ax0.tick_params(labelsize=16)

ax1 = fig.add_subplot(spec[1])
ax1.grid(axis='x')
ax1.set_xlim(0.3,8.5)

ax1.tick_params(      # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    top=False,         # ticks along the top edge are off
    left=False,      # ticks along the left edge are off
    labelleft=False) # labels along the left edge are off

ax1.set_xticks(xlocs0)
ax1.set_xticklabels(np.arange(0,len(objects0),1),fontsize=16)
for i in range(len(objects0)):
    ax1.scatter(x[objects0[i]],y[objects0[i]],s=10,c=['r'])
for i in range(len(objects1)):
    ax1.scatter(x[objects1[i]],y[objects1[i]],s=10,c=['k'])

custom_lines = [Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['Taken out'],loc='upper left',fontsize=14)

if perarea:
    plt.savefig('Figures/tvf-obscomp-wave-perarea')
else:
    plt.savefig('Figures/tvf-obscomp-wave')

In [32]:
fig = plt.figure(figsize=(18,7.5))

heights = [3, 1]
spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=heights, hspace=0.25)

ax0 = fig.add_subplot(spec[0])
ax0.grid()

xlocs0 = np.zeros(len(objects0))
for i in range(len(objects0)):
    xlocs0[i] = np.mean(x[objects0[i]])
    
xlocs1 = np.zeros(len(objects1))
for i in range(len(objects1)):
    xlocs1[i] = np.mean(x[objects1[i]])
    
ax0.bar(xlocs0,tvf[0],width=0.15,label=foldernames[0],color=my_cmap(0))
ax0.bar(xlocs1+0.05,tvf[1,labellist],width=0.15,label=foldernames[1],color=my_cmap(1))
ax0.set_axisbelow(True)

ax0.set_xticks(xlocs0)
ax0.set_xticklabels(np.arange(0,len(objects0),1))
ax0.set_xlim(0.3,8.5)
ax0.set_xlabel("ending object",fontsize=18)
if perarea:
    ax0.set_ylabel("Volume flux [$m$ $s^{-1}$]",fontsize = 18)
else:
    ax0.set_ylabel("$m^2$ $s^{-1}$",fontsize = 18)
ax0.legend(fontsize = 18)
ax0.tick_params(labelsize=16)

ax1 = fig.add_subplot(spec[1])
ax1.grid(axis='x')
ax1.set_xlim(0.3,8.5)

ax1.tick_params(      # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    top=False,         # ticks along the top edge are off
    left=False,      # ticks along the left edge are off
    labelleft=False) # labels along the left edge are off

ax1.set_xticks(xlocs0)
ax1.set_xticklabels(np.arange(0,len(objects0),1),fontsize=16)
for i in range(len(objects0)):
    ax1.scatter(x[objects0[i]],y[objects0[i]],s=10,c=['r'])
for i in range(len(objects1)):
    ax1.scatter(x[objects1[i]],y[objects1[i]],s=10,c=['k'])

custom_lines = [Line2D([0], [0], color='r', lw=4)]
ax1.legend(custom_lines, ['Taken out'],loc='upper left',fontsize=14)

if perarea:
    plt.savefig('Figures/tvf-obscomp-perarea')
else:
    plt.savefig('Figures/tvf-obscomp')